In [1]:
import pandas as pd

In [2]:
df_eng_events = pd.read_csv('/Users/zach/DATA 511 Jupyter/data1/events_England.csv')
df_eng_matches = pd.read_csv('/Users/zach/DATA 511 Jupyter/data1/matches_England.csv')
df_teams = pd.read_csv('/Users/zach/DATA 511 Jupyter/data1/teams.csv')
df_players = pd.read_csv('/Users/zach/DATA 511 Jupyter/data1/player_games.csv')
df_actions = pd.read_csv('/Users/zach/DATA 511 Jupyter/data1/actions.csv')

In [3]:
df_events_teams_matches = pd.merge(df_eng_events, df_eng_matches[['wyId', 'team1.teamId','team2.teamId','team1.score','team2.score']], left_on = 'matchId', right_on = 'wyId',how='left')
df_events_teams_matches = df_events_teams_matches.query('eventId == 8 | eventId == 10')
for idx, row in df_events_teams_matches.iterrows():
    if df_events_teams_matches.loc[idx, 'teamId'] == df_events_teams_matches.loc[idx, 'team1.teamId']:
        df_events_teams_matches.loc[idx, 'team'] = df_events_teams_matches.loc[idx, 'team1.teamId']
        df_events_teams_matches.loc[idx, 'opponent'] = df_events_teams_matches.loc[idx, 'team2.teamId']
        df_events_teams_matches.loc[idx, 'goals_f'] = df_events_teams_matches.loc[idx, 'team1.score']
        df_events_teams_matches.loc[idx, 'goals_a'] = df_events_teams_matches.loc[idx, 'team2.score']
    else:
        df_events_teams_matches.loc[idx, 'team'] = df_events_teams_matches.loc[idx, 'team2.teamId']
        df_events_teams_matches.loc[idx, 'opponent'] = df_events_teams_matches.loc[idx, 'team1.teamId']
        df_events_teams_matches.loc[idx, 'goals_f'] = df_events_teams_matches.loc[idx, 'team2.score']
        df_events_teams_matches.loc[idx, 'goals_a'] = df_events_teams_matches.loc[idx, 'team1.score']
df_events_teams_matches = pd.merge(df_events_teams_matches, df_teams[['wyId','name']], left_on = 'team', right_on = 'wyId',how='left')
df_events_teams_matches = pd.merge(df_events_teams_matches, df_teams[['wyId','name']], left_on = 'opponent', right_on = 'wyId',how='left')


In [4]:
df_players = df_players.drop_duplicates(subset=['player_id'])
df_passes = pd.merge(df_events_teams_matches, df_players[['player_id','player_name']], left_on='playerId', right_on='player_id',how='left')


In [5]:
df_passes = pd.merge(df_passes, df_actions[['original_event_id','start_x', 'start_y', 'end_x', 'end_y','result_name']], left_on='id', right_on='original_event_id',how='left')
df_passes = df_passes.rename(columns={'name_x':'team_name','name_y':'opp_name'})
df_passes = df_passes.drop(['eventId', 'tags', 'positions',
       'eventName', 'teamId', 'subEventId',
       'pos_orig_y', 'pos_orig_x', 'pos_dest_y', 'pos_dest_x',
       'wyId_x', 'team1.teamId', 'team2.teamId', 'team1.score', 'team2.score',
       'wyId_y', 'wyId','id','player_id','original_event_id','team','opponent'], axis=1)


In [ ]:
df_passes.to_csv(r'/Users/zach/DATA 511 Jupyter/data/initial_clean_and_join.csv',index=False)

In [4]:
df_passes = pd.read_csv('/Users/zach/DATA 511 Jupyter/data/passes2.csv')

68.0

In [6]:
display(df_passes.columns)

Index(['subEventName', 'playerId', 'matchId', 'matchPeriod', 'eventSec',
       'tagsList', 'goals_f', 'goals_a', 'team_name', 'opp_name',
       'player_name', 'start_x', 'start_y', 'end_x', 'end_y', 'result_name'],
      dtype='object')

In [9]:
df_pass_pbp = df_passes.sort_values(by = ['matchId', 'matchPeriod', 'eventSec'], ascending = [True, True, True])

In [44]:
prev = 'success'
play = 1
for idx, row in df_pass_pbp.iterrows():
    if idx == 0:
        df_pass_pbp.loc[idx, 'play'] = play
        df_pass_pbp.loc[idx, 'possession'] = 'start'
    else:
        if df_pass_pbp.loc[idx, 'matchId'] == df_pass_pbp.loc[idx-1, 'matchId']:
            if df_pass_pbp.loc[idx, 'subEventName'] == 'Shot':
                if prev == 'success' and df_pass_pbp.loc[idx, 'team_name'] == df_pass_pbp.loc[idx-1, 'team_name']:
                    df_pass_pbp.loc[idx, 'play'] = play
                    play = play + 1
                    df_pass_pbp.loc[idx, 'possession'] = 'end'
                else:
                    play = play + 1
                    df_pass_pbp.loc[idx, 'play'] = play
                    play = play + 1
                    df_pass_pbp.loc[idx, 'possession'] = 'end'
            else:
                if idx != len(df_pass_pbp.index)-1:
                    if prev == 'success' and df_pass_pbp.loc[idx, 'team_name'] == df_pass_pbp.loc[idx-1, 'team_name']:
                        df_pass_pbp.loc[idx, 'play'] = play
                        if df_pass_pbp.loc[idx, 'result_name'] == 'fail' or df_pass_pbp.loc[idx, 'team_name'] != df_pass_pbp.loc[idx+1, 'team_name']:
                            df_pass_pbp.loc[idx, 'possession'] = 'end'
                        else:
                            df_pass_pbp.loc[idx, 'possession'] = 'middle'

                    else:
                        if df_pass_pbp.loc[idx-1, 'subEventName'] == 'Shot':
                            df_pass_pbp.loc[idx, 'play'] = play
                            df_pass_pbp.loc[idx, 'possession'] = 'start'
                        else:
                            play = play + 1
                            df_pass_pbp.loc[idx, 'play'] = play
                            df_pass_pbp.loc[idx, 'possession'] = 'start'
                else:
                    if prev == 'success' and df_pass_pbp.loc[idx, 'team_name'] == df_pass_pbp.loc[idx-1, 'team_name']:
                        df_pass_pbp.loc[idx, 'play'] = play
                        if df_pass_pbp.loc[idx, 'result_name'] == 'fail':
                            df_pass_pbp.loc[idx, 'possession'] = 'end'
                        else:
                            df_pass_pbp.loc[idx, 'possession'] = 'middle'

                    else:
                        if df_pass_pbp.loc[idx-1, 'subEventName'] == 'Shot':
                            df_pass_pbp.loc[idx, 'play'] = play
                            df_pass_pbp.loc[idx, 'possession'] = 'start'
                        else:
                            play = play + 1
                            df_pass_pbp.loc[idx, 'play'] = play
                            df_pass_pbp.loc[idx, 'possession'] = 'start'
        else:
            if df_pass_pbp.loc[idx-1, 'subEventName'] == 'Shot':
                df_pass_pbp.loc[idx, 'play'] = play
                df_pass_pbp.loc[idx, 'possession'] = 'start'
            else:
                play = play + 1
                df_pass_pbp.loc[idx, 'play'] = play
                df_pass_pbp.loc[idx, 'possession'] = 'start'
            
        prev = df_pass_pbp.loc[idx, 'result_name']


In [17]:
display(df_pass_pbp)

,subEventName,playerId,matchId,matchPeriod,eventSec,tagsList,goals_f,goals_a,team_name,opp_name,player_name,start_x,start_y,end_x,end_y,result_name,play,possession
0,Simple pass,25413,2499719,1H,2.758649,[1801],4.0,3.0,Arsenal,Leicester City,Alexandre Lacazette,53.55,33.32,72.45,53.04,success,1.0,start
1,High pass,370224,2499719,1H,4.946850,[1801],4.0,3.0,Arsenal,Leicester City,Rob Holding,72.45,53.04,51.45,51.00,success,1.0,NaN
2,Head pass,3319,2499719,1H,6.542188,[1801],4.0,3.0,Arsenal,Leicester City,Mesut Özil,51.45,51.00,68.25,48.28,success,1.0,NaN
3,Head pass,120339,2499719,1H,8.143395,[1801],4.0,3.0,Arsenal,Leicester City,Mohamed Naser Elsayed Elneny,68.25,48.28,61.95,64.60,success,1.0,NaN
4,Simple pass,167145,2499719,1H,10.302366,[1801],4.0,3.0,Arsenal,Leicester City,Héctor Bellerín Moruno,61.95,64.60,29.40,59.84,success,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351084,Smart pass,9194,2500098,2H,2614.478054,[1801],1.0,3.0,Everton,West Ham United,Yannick Bolasie Yala,61.95,67.32,35.70,46.92,success,380.0,NaN
351085,Simple pass,145692,2500098,2H,2615.493725,[1801],1.0,3.0,Everton,West Ham United,El-Hadji Baye Oumar Niasse,35.70,46.92,37.80,57.80,success,380.0,NaN
351086,High pass,56,2500098,2H,2616.871291,[1801],1.0,3.0,Everton,West Ham United,Davy Klaassen,37.80,57.80,32.55,6.80,success,380.0,NaN
351087,Simple pass,8239,2500098,2H,2625.882140,[1801],1.0,3.0,Everton,West Ham United,Leighton Baines,32.55,6.80,32.55,12.92,success,380.0,NaN


In [45]:
df_pass_pbp.to_csv(r'/Users/zach/DATA 511 Jupyter/data1/pbp.csv',index=False)